In [1]:
import os
import langchain
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
model = ChatGroq(model="Gemma2-9b-It", api_key=groq_api_key)

In [5]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi My name is Sachin")])

AIMessage(content="Hi Sachin, it's nice to meet you! 👋\n\nWhat can I do for you today? 😄 \n", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 19, 'total_tokens': 45, 'completion_time': 0.052, 'prompt_time': 0.002295239, 'queue_time': None, 'total_time': 0.054295238999999995}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ecd48364-0c2d-4dfe-a8e7-2e2d35a8455b-0', usage_metadata={'input_tokens': 19, 'output_tokens': 26, 'total_tokens': 45})

In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You told me your name is Krish and that you are a Chief AI Engineer!  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm happy to chat. 😊 \n", response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 102, 'total_tokens': 150, 'completion_time': 0.101901188, 'prompt_time': 0.016712218, 'queue_time': None, 'total_time': 0.118613406}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-83c5e3dc-3f9c-456c-8c3a-fd5622bcc94b-0', usage_metadata={'input_tokens': 102, 'output_tokens': 48, 'total_tokens': 150})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]


with_message_history=RunnableWithMessageHistory(model, get_session_history)


In [8]:
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
| RunnableBranch(branches=[(RunnableBinding(bound=RunnableLambda(_is_not_async), config={'run_name': 'RunnableWithMessageHistoryInAsyncMode'}), RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10e99f0d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10f2682e0>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********')), config_factories=[<function Runnable.with_alisteners.<locals>.<lambda> at 0x10ec6e950>]))], default=RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10e99f0d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10f2682e0>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********')), config_factories=[<function Runnable.with_listeners.<locals>.<lambda> at 0x10f653520>])), 

In [13]:
config = {"configurable":{"session_id":"class1"}}

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

In [15]:
response

AIMessage(content="Hi Krish, it's nice to meet you!  That's a fascinating role. \n\nWhat are you working on these days that you're excited about?  I'm always interested to learn about the latest advancements in AI.\n", response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 27, 'total_tokens': 80, 'completion_time': 0.11321672, 'prompt_time': 0.007132154, 'queue_time': None, 'total_time': 0.12034887400000001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e29f1c7b-eac9-41f2-8de3-bbe505951b66-0', usage_metadata={'input_tokens': 27, 'output_tokens': 53, 'total_tokens': 80})

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.